<a href="https://colab.research.google.com/github/archaeogeek/colab/blob/main/NLP_with_Spacy_WIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting up working environment.
Book says spaCy 2.0 or later

In [ ]:
!pip install spacy
!pip install pytextrank --upgrade
!python -m spacy info

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-24 18:05:52.654728: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

============================== Info about spaCy ==============================

spaCy version    3.4.4                         
Location         /usr/local/lib/python3.8/dist-packages/spacy
Platform         Linux-5.10.147+-x86_64-with-glibc2.29
Python version   3.8.10                        
Pipelines        en_core_web_sm (3.4.1)        



Install statistical models for spaCy. Models have naming syntax lang_type_genre_size. Downloading 'en' downloads the default en_core_web_md, which is the smallest general purpose model for the English language

In [ ]:
!python -m spacy download en

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-19 11:42:25.620464: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 69.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Basic NLP operations

**Tokenisation**
Parsing text into tokens, which can be words, numbers or punctuation


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(u'I am driving to Kendal')
print([w.text for w in doc])

['I', 'am', 'driving', 'to', 'Kendal']


**Lemmatization**
The base form of any given token. eg, the Lemma for 'driving' is 'drive'. Should also work for tenses

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(u'this dog enjoyed both running and sleeping, in fact it slept very well and runs most days')
for token in doc:
  print(token.text, token.lemma_)

this this
dog dog
enjoyed enjoy
both both
running run
and and
sleeping sleep
, ,
in in
fact fact
it it
slept sleep
very very
well well
and and
runs run
most most
days day


We need to deal with nicknames somehow- we can define them for spaCy to use. It doesn't seem to work if you just call 'doc' in the final line though- you seem to need to call it in the print statement. Maybe you need to define it after you add the tokenizer special case?

In [ ]:
import spacy
from spacy.symbols import ORTH, LEMMA
nlp = spacy.load('en_core_web_sm')
doc = nlp(u'I am driving to Lancaster, in Lancs')
print([w.text for w in doc])
special_case = [{ORTH: u'Lancs', LEMMA: u'Lancashire'}]
nlp.tokenizer.add_special_case(u'Lancs', special_case)
print([w.lemma_ for w in nlp(u'I am driving to Lancaster in Lancs')])

['I', 'am', 'driving', 'to', 'Lancaster', ',', 'in', 'Lancs']


ValueError: ignored

**Part-of-speech Tagging**
Tags each word in a sentence with whether it's a noun, verb etc. Standard grammatical constructs, along with punctuation are called coarse-grained parts of speech. Tenses and types of pronoun etc are fine-grained parts of speech. NLP needs to do this for multi-sentence phrases as lemmatization will reduce everything to it's basic form- tense and anything signifying intent will disappear. The following snippet identifies the token that is in the *present progressive* form (`VBG`) or *base* form (`VB`) to identify the part of the phrase signifying intent rather than something that happened in the past.

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(u'I have driven to Lancaster. Now I am driving to Kendal')
print([w.text for w in doc if w.tag_ == 'VBG' or w.tag_ == 'VB'])
print([w.text for w in doc if w.pos_ == 'PROPN'])

['driving']
['Lancaster', 'Kendal']


**Syntactic Relations** The next step is to link the proper noun with verb that best signifies intent. SpaCy uses syntactic dependency labels to show the relationship between pairs of words in a sentence. Verb phrases are parents, noun phrases are children (but can have children of their own). The relationship between parents or `heads`, and children is one-to-many, so the dependency label (`dobj`) is always assigned to the child



In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(u'I have driven to Lancaster. Now I am walking to Liverpool')
for token in doc:
  print(token.head.text, token.text, token.pos_, token.dep_)

driven I PRON nsubj
driven have AUX aux
driven driven VERB ROOT
driven to ADP prep
to Lancaster PROPN pobj
driven . PUNCT punct
walking Now ADV advmod
walking I PRON nsubj
walking am AUX aux
walking walking VERB ROOT
walking to ADP prep
to Liverpool PROPN pobj


In the above, looking at the `token.head.text` and the `token.dep_` you can extract the `ROOT` and the `pobj` to extract the intent part of the phrase. We can also split the phrase up into sentences with `doc.sents`

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(u'I have driven to Lancaster. Now I am cycling to Liverpool')
for sent in doc.sents:
  print([w.text for w in sent if w.dep_ == 'ROOT' or w.dep_ == 'pobj'])

['driven', 'Lancaster']
['cycling', 'Liverpool']


**Named Entity Recognition** named entities are real objects such as people, organisations, locations. 

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(u'I have driven to Lancaster. Now I am walking to Galgate. I have been to Burton-in-Kendal too.')
for token in doc:
  if token.ent_type !=0:
    print(token.text, token.ent_type_)

Lancaster GPE
Galgate ORG
Burton ORG


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(u'I have driven to Lancaster. Now I am walking to Galgate. I have been to Horndean and Burton-in-Kendal too.')
for token in doc:
  if token.ent_type !=0:
    print(token.text, token.ent_type_)

Lancaster GPE
Galgate ORG
Horndean LOC
Burton PERSON


# Gotchas

**UK Placenames** If we try a small village with a multi-part name such as *Burton-in-Kendal*, spaCy doesn't recognise it as a single proper noun. So we're going to need to try different entity taggers or dictionaries to find one that can understand more UK places (could be hills, mountains, etc too). If it's a single small place such as *Galgate* that's not in the list, it recognises it as a proper noun (`PROPN`) but doesn't tag it as the object (`pobj`). Then... if we use named entity recognition, it mis-tags *Lancaster* and *Burton* as `PERSON`, which is funny but problematic

This might be the point at which we need to use something like [mordecai](https://github.com/openeventdata/mordecai) to do our geoparsing, which will require an ES install...

Alternative approach- based on Chapter 10 (Training Models), there is an example that corrects the tag for a location from LOC to GPE. The section "Automating the Example Creation Process" takes a text file and a list of entities, and says "for each entity in this list that appears in the text file, classify as GPE". We should be able to both the text file and the list from the OS Open Names data and then use that to train the model. Let's see if we can use this approach to add Lancaster as a GPE in the model though...

In [ ]:
import spacy
from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER
from spacy.lang.char_classes import CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex

nlp = spacy.load('en_core_web_sm')

# Modify tokenizer infix patterns to stop the tokeniser breaking on hyphens
# from https://spacy.io/usage/linguistic-features#native-tokenizers
infixes = (
    LIST_ELLIPSES
    + LIST_ICONS
    + [
        r"(?<=[0-9])[+\\-\\*^](?=[0-9-])",
        r"(?<=[{al}{q}])\\.(?=[{au}{q}])".format(
            al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
        ),
        r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
        # ✅ Commented out regex that splits on hyphens between letters:
        # r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
        r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
    ]
)

infix_re = compile_infix_regex(infixes)
nlp.tokenizer.infix_finditer = infix_re.finditer

doc = nlp(u'Could you send a taxi to Horndean? I need to get to Lancaster, via Burton-in-Kendal, to meet Richard Burton')
train_exams = []
places = ['Horndean', 'Burton-in-Kendal', 'Lancaster']
for sent in doc.sents:
  entities = []
  for token in sent:
    # sanity check the existing token type and type number before we process
    print(token.text, token.ent_type_, token.ent_type)
    if token.text in places:
      start = token.idx - sent.start_char
      # force entries from the list to have an entity type of GPE
      # note this doesn't affect the type number which could still be zero
      entity = (start, start + len(token), 'GPE')
    else:
      start = token.idx - sent.start_char
      # other entries not in the places list get their existing type and type number
      entity = (start, start + len(token), token.ent_type_)
    # prevent adding all the entities that still have no entity type
    if entity[2] !='':
      entities.append(entity)
  tpl = (sent.text, {'entities': entities})
  train_exams.append(tpl)
# print resulting correctly tagged training examples
print(train_exams)

Could  0
you  0
send  0
a  0
taxi  0
to  0
Horndean LOC 385
?  0
I  0
need  0
to  0
get  0
to  0
Lancaster PERSON 380
,  0
via  0
Burton-in-Kendal  0
,  0
to  0
meet  0
Richard PERSON 380
Burton PERSON 380
[('Could you send a taxi to Horndean?', {'entities': [(25, 33, 'GPE')]}), ('I need to get to Lancaster, via Burton-in-Kendal, to meet Richard Burton', {'entities': [(17, 26, 'GPE'), (32, 48, 'GPE'), (58, 65, 'PERSON'), (66, 72, 'PERSON')]})]


The above approach seems to correctly tag the places and doesn't add all the other scrote to the list of training examples

# Spacy Tokenisation


Aka how we deal with things like "London Road" without splitting it into "London" and "Road"

The example below shows using `token.rights` and `token.lefts` to pick up the left and right children of a token

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("I live on Greaves Road in Lancaster but I used to live on Greenfield Street") #"bright red apples on the tree"
for sent in doc.sents:
  for token in sent:
    print(token.text, token.ent_type_, [t.text for t in token.lefts], token.n_lefts, [t.text for t in token.rights], token.n_rights)
# print([token.text for token in doc[2].lefts])  # ['bright', 'red']
# print([token.text for token in doc[3].rights])  # ['on']
print(doc[2].n_lefts)  # 2
print(doc[2].n_rights)  # 1

I  [] 0 [] 0
live  ['I'] 1 ['on', 'but', 'used'] 3
on  [] 0 ['Road'] 1
Greaves FAC [] 0 [] 0
Road FAC ['Greaves'] 1 ['in'] 1
in  [] 0 ['Lancaster'] 1
Lancaster GPE [] 0 [] 0
but  [] 0 [] 0
I  [] 0 [] 0
used  ['I'] 1 ['live'] 1
to  [] 0 [] 0
live  ['to'] 1 ['on'] 1
on  [] 0 ['Street'] 1
Greenfield FAC [] 0 [] 0
Street FAC ['Greenfield'] 1 [] 0
0
1


https://stackoverflow.com/questions/57206701/spacy-tokenizer-rule-for-exceptions-that-contain-whitespace suggests a couple of approaches. The simplest might be to merge noun chunks?

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("merge_noun_chunks")
doc = nlp(u'I live on Greaves Road')
for token in doc:
  print(token.text, token.pos_, token.dep_)


I PRON nsubj
live VERB ROOT
on ADP prep
Greaves Road PROPN pobj


# Adding a terminology list to our model

Thanks to https://notebook.community/psychemedia/parlihacks/notebooks/Text%20Scraping%20-%20Notes this might finally be the approach for pre-loading our additional location entries! The next step is (I think) to turn this into a custom pipeline that we can load alongside the proper one rather than doing the list munging each time

In [6]:
import spacy
import pandas as pd
from spacy import displacy
from spacy.pipeline import EntityRuler


nlp = spacy.load("en_core_web_sm")
# make sure we load and run our custom pipeline before the standard one
# else our entities get overwritten
ruler = nlp.add_pipe("entity_ruler", before="ner")

# make a set of patterns from the csv
# and then add them to our custom pipeline
# note this relies on top50_list.csv being loaded into the notebook session
opennames = pd.read_csv('top50_list.csv')
#opennames = pd.read_csv('osopennames_singlecolumn_2.csv', nrows=50)
opennames_list= opennames["THING"].tolist()
#opennames_list= opennames.iloc[:,1].to_list()
patterns=[]
for t in opennames_list:
  patterns.append({"label":"GPE", "pattern":t})
print(len(patterns))
ruler.add_patterns(patterns)

# test with a doc that has some entries from the list
# and some that are not
# and use displacy to render the text showing the entities
doc = nlp("BN21 1AA is in my csv along with Saxon Ground, St Aubyn's Road and A2040. London is not, and Richard Burton is certainly not!")
displacy.render(doc, jupyter=True, style='ent')


50


Next we need to test this out with the whole csv, which has now been committed to the github repository at https://github.com/archaeogeek/colab, so let's download it, check which folder we're in and that it has the right sort of data in it

In [ ]:
!wget https://raw.githubusercontent.com/archaeogeek/colab/master/osopennames_singlecolumn_2.csv
!pwd
!head -n 10 osopennames_singlecolumn_2.csv

Saving the new model to disk

In [1]:
import spacy
import pandas as pd
from spacy import displacy
from spacy.pipeline import EntityRuler


nlp = spacy.load("en_core_web_sm", enable=["ner"])
# make sure we load and run our custom pipeline before the standard one
# else our entities get overwritten
ruler = nlp.add_pipe("entity_ruler", before="ner")

patterns=[]
#for i in range(0,3000000,1000000):
  # add osopennames in chunks
#osopennames = pd.read_csv('osopennames_singlecolumn_2.csv', nrows=1000000, skiprows=i)
osopennames = pd.read_csv('osopennames_singlecolumn_2.csv')
o_list = osopennames.iloc[:,1]                        
for t in o_list:
  patterns.append({"label":"GPE", "pattern":t})                     
print(len(patterns))
ruler.add_patterns(patterns)
nlp.to_disk("osopennames_ner")

# test with a doc that has some entries from the list
# and some that are not
# and use displacy to render the text showing the entities
#doc = nlp("Gloup is in my csv along with Westing. ZE2 9EH is in my csv along with Saxon Ground, Doos' Geo and A2040. Los Angeles is not, and Richard Burton is certainly not!")
#displacy.render(doc, jupyter=True, style='ent')



/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


2973899


Loading our custom model and actually using it. We're almost there but have to figure out how to re-add the `en_core_web_sm` model *and* our custom pipeline

In [1]:
import spacy
from spacy import displacy
nlp = spacy.load("osopennames_ner")
print(nlp.pipe_names)
doc = nlp("Gloup is in my csv along with Westing. ZE2 9EH is in my csv along with Saxon Ground, Doos' Geo and A2040. Los Angeles is not, and Richard Burton is certainly not!")
displacy.render(doc, jupyter=True, style='ent')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


['entity_ruler', 'ner']
